In [1]:
using LinearAlgebra
using JuMP
using GLPK
using XLSX
using Complementarity #not sure I need this one?
using PATHSolver
using Revise #not having to restart sessions
#using Pandas #optional (only for EXIOBASE data)

include("../src/SUT_structure.jl") # Used for the SUT setup <sut = SUT.structure(...)>
include("../src/Constructs.jl") # Used to derive single-production systems from the SUT setup, e.g. as <itc = Constructs.ITC(sut)>
include("../src/Auxiliary.jl") # Includes some helper functions
include("../src/Model_data.jl") # Sets up the data structure for RCOT modelling based on SUT.structure or Constructs.construct
include("../src/RCOT_model.jl"); # Builds and solves the RCOT models

In [2]:
# Load the workbook
xf = XLSX.readxlsx("../data/SUT_c+i-.xlsx")

XLSXFile("../data/SUT_c+i-.xlsx") containing 4 Worksheets
            sheetname size          range        
-------------------------------------------------
                    V 4x6           A1:F4        
                  U+e 6x5           A1:E6        
                    F 5x4           A1:D5        
                   pi 5x2           A1:B5        


In [3]:
# Get data from all worksheets and convert it to float64 data type
V = convert(Matrix{Float64}, xf["V!B2:F4"])
U = convert(Matrix{Float64}, xf["U+e!B2:D6"])
e = convert(Matrix{Float64}, xf["U+e!E2:E6"])
F = convert(Matrix{Float64}, xf["F!B2:D5"])
pii = convert(Matrix{Float64}, xf["pi!B2:B5"])
t = xf["V!A1"];

In [16]:
# Load the workbook
xf = XLSX.readxlsx("../data/SUT_c=i.xlsx")

XLSXFile("../data/SUT_c=i.xlsx") containing 4 Worksheets
            sheetname size          range        
-------------------------------------------------
                    V 4x4           A1:D4        
                  U+e 4x5           A1:E4        
                    F 5x4           A1:D5        
                   pi 5x2           A1:B5        


In [17]:
# Get data from all worksheets and convert it to float64 data type
V = convert(Matrix{Float64}, xf["V!B2:D4"])
U = convert(Matrix{Float64}, xf["U+e!B2:D4"])
e = convert(Matrix{Float64}, xf["U+e!E2:E4"])
F = convert(Matrix{Float64}, xf["F!B2:D5"])
pii = convert(Matrix{Float64}, xf["pi!B2:B5"])
t = xf["V!A1"]

"mixed"

In [18]:
sut = SUT.structure(U,V,F,e,pii,t);

[ Info: The SUT system is not given in monetary units such that no total industry output and dependent matrices may be calculated.
[ Info: A structure for supply-use elements was set up. Changing individual elements will not change others automatically.


In [19]:
model_data = Model_data.SU(sut);

[ Info: You are setting up an SU model dataset. Elements of this dataset are now treated independently, meaning that no recalculation whatsoever takes place when individual elements are changed.


In [35]:
ctc = Constructs.CTC(sut)

construct([113.0; 74.0; 127.0;;], [26.0; 20.0; 68.0;;], [16.74074074074074 -4.611620795107033 74.87088005436628; 53.01234567901234 -6.7655453618756365 7.753199682863292; 0.0 59.0 0.0], [0.14814814814814814 -0.06231919993387883 0.5895344886170574; 0.4691358024691358 -0.0914262886739951 0.06104881640049836; 0.0 0.7972972972972974 0.0], [1.496064824981334 0.5849598137818962 0.9176929158065789; 0.6730818217752205 1.2221748869076374 0.4714172778778162; 0.5366463173613245 0.9744367341560893 1.3758597215512318], [13.950617283950617 12.612640163098877 7.436742552950504; 5.580246913580247 -4.760448521916412 24.180201608336162; 20.925925925925924 6.152905198776758 -1.078831124702684; 2.7901234567901234 12.372069317023445 13.83780722618643], [0.12345679012345678 0.17044108328511998 0.058557027975988225; 0.04938271604938271 -0.06433038543130287 0.1903952882546155; 0.18518518518518517 0.08314736755103729 -0.008494733265375467; 0.024691358024691357 0.16719012590572224 0.108959112017216], [34.0; 24.9

In [46]:
model_data = Model_data.IO(ctc);

[ Info: You are setting up an IO model dataset. Elements of this dataset are now treated independently, meaning that no recalculation whatsoever takes place when individual elements are changed.


In [7]:
using Ipopt

In [23]:
# Heckscher-Ohlin - Quadratic program
# behaves diverging, no real solution
# probably because I assumed not-given commodity prices, but in the paper they are in fact given
#based on https://www.sciencedirect.com/science/article/pii/S0014292177800330/pdf
# Instantiate the model
primal = Model(Ipopt.Optimizer)
(@isdefined primal) && (primal = nothing; primal = Model(Ipopt.Optimizer));
# Define the optimisation model
@variable(primal, x_star[1:size(itc.A,1)] >= 0, base_name = "x*")
@variable(primal, p_star[1:size(itc.A,2)] >= 0, base_name = "p*")#Not sur eif p is actually unbounded; the paper doesn't set any bounds at least
@objective(primal, Max, sum(p_star'*x_star))
@constraint(primal, c1, itc.R*x_star .<= itc.ϕ);
# Run the model and show results
optimize!(primal)
@show termination_status(primal)
@show primal_status(primal)
@show dual_status(primal)
@show objective_value(primal)
@show value.(x_star)
@show value.(p_star)
@show value.(c1)
@show shadow_price.(c1)

This is Ipopt version 3.14.13, running with linear solver MUMPS 5.6.0.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       16
Number of nonzeros in Lagrangian Hessian.............:        5

Total number of variables............................:       10
                     variables with only lower bounds:       10
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        4
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        4

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  4.9999900e-04 0.00e+00 1.01e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

4×1 Matrix{Float64}:
    3.132960002017056e20
    4.062768677476948e20
 3173.784092736588
    3.5236770732592746e20

In [16]:
itc.ϕ

4×1 Matrix{Float64}:
 34.16666666666667
 16.11111111111111
  2.2222222222222223
 30.476190476190474

In [15]:
# Heckscher-Ohlin - Quadratic program
# Instantiate the model
dual = Model(Ipopt.Optimizer)
(@isdefined dual) && (dual = nothing; dual = Model(Ipopt.Optimizer));
# Define the optimisation model
@variable(dual, pii_star[1:size(itc.R,1)] >= 0, base_name = "pii*")
@variable(dual, p_star[1:size(itc.A,2)] >= 0, base_name = "p*")
@objective(dual, Min, sum(pii_star'*itc.ϕ))
@constraint(dual, c1, itc.R'*pii_star .>= p_star);
# Run the model and show results
optimize!(dual)
@show termination_status(dual)
@show primal_status(dual)
@show dual_status(dual)
@show objective_value(dual)
@show value.(x_star)
@show value.(p_star)
@show value.(c1)
@show shadow_price.(c1)

This is Ipopt version 3.14.13, running with linear solver MUMPS 5.6.0.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       21
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        9
                     variables with only lower bounds:        9
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        5
        inequality constraints with only lower bounds:        5
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  8.2976107e-01 7.12e-03 3.13e+01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

5-element Vector{Float64}:
 -0.3036538296955946
 -0.32241290617638896
 -0.2998028334535814
 -0.2998028334535814
 -0.2998028334535814

In [27]:
itc.x

5×1 Matrix{Float64}:
 88.0
 95.0
 76.0
 10.0
 10.0

In [24]:
# Heckscher-Ohlin - Quadratic program.
# In fact not quadratic when prices are given...
#based on https://www.sciencedirect.com/science/article/pii/S0014292177800330/pdf
# Instantiate the model
primal = Model(Ipopt.Optimizer)
(@isdefined primal) && (primal = nothing; primal = Model(Ipopt.Optimizer));
# Define the optimisation model
@variable(primal, x_star[1:size(itc.A,1)] >= 0, base_name = "x*")
#@variable(primal, p_star[1:size(itc.A,2)] >= 0, base_name = "p*")#Not sur eif p is actually unbounded; the paper doesn't set any bounds at least
p_star = ones(size(itc.A,1))
@objective(primal, Max, sum(p_star'x_star))
@constraint(primal, c1, itc.R*x_star .<= itc.ϕ);
# Run the model and show results
optimize!(primal)
@show termination_status(primal)
@show primal_status(primal)
@show dual_status(primal)
@show objective_value(primal)
@show value.(x_star)
@show value.(p_star)
@show value.(c1)
@show shadow_price.(c1)

This is Ipopt version 3.14.13, running with linear solver MUMPS 5.6.0.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       16
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        5
                     variables with only lower bounds:        5
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        4
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        4

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  4.9999950e-02 0.00e+00 1.52e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

4×1 Matrix{Float64}:
 3.4788606819335546
 2.405752127193995
 3.422497616266235
 3.7332105768477866

In [28]:
# Heckscher-Ohlin - Quadratic program
# Instantiate the model
dual = Model(Ipopt.Optimizer)
(@isdefined dual) && (dual = nothing; dual = Model(Ipopt.Optimizer));
# Define the optimisation model
@variable(dual, pii_star[1:size(itc.R,1)] >= 0, base_name = "pii*")
#@variable(dual, p_star[1:size(itc.A,2)] >= 0, base_name = "p*")
p_star = ones(size(itc.A,1))
@objective(dual, Min, sum(pii_star'*itc.ϕ))
@constraint(dual, c1, itc.R'*pii_star .>= p_star);
# Run the model and show results
optimize!(dual)
@show termination_status(dual)
@show primal_status(dual)
@show dual_status(dual)
@show objective_value(dual)
@show value.(x_star)
@show value.(p_star)
@show value.(c1)
@show shadow_price.(c1)

This is Ipopt version 3.14.13, running with linear solver MUMPS 5.6.0.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       16
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        4
                     variables with only lower bounds:        4
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        5
        inequality constraints with only lower bounds:        5
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  8.2976107e-01 9.97e-01 2.99e+01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

5-element Vector{Float64}:
 -88.00000001809542
 -94.99999997554298
 -31.99999999877881
 -31.99999999877881
 -31.999999998780645

In [ ]:
# SU-NEO - LP
# Instantiate the model
primal = Model(GLPK.Optimizer)
(@isdefined primal) && (primal = nothing; primal = Model(GLPK.Optimizer));
# Define the optimisation model
@variable(primal, z_star[1:size(V,1)] >= 0, base_name = "z*")
@variable(primal, c_star >= 0, base_name = "c*")
@objective(primal, Max, c_star)
@constraint(primal, c1, (model_data.V'-model_data.U)*(z_star) .>= c_star*model_data.e)
@constraint(primal, c2, model_data.F*z_star .<= model_data.ϕ);
# Run the model and show results
optimize!(primal)
@show termination_status(primal)
@show primal_status(primal)
@show dual_status(primal)
@show objective_value(primal)
@show value.(z_star)
@show value.(c_star)
@show value.(c1)
@show value.(c2)
@show shadow_price.(c1)
@show shadow_price.(c2)

termination_status(primal) = MathOptInterface.OPTIMAL
primal_status(primal) = MathOptInterface.FEASIBLE_POINT
dual_status(primal) = MathOptInterface.FEASIBLE_POINT
objective_value(primal) = 1.0000000000000004
value.(z_star) = [1.0000000000000002, 1.0, 1.0000000000000002]
value.(c_star) = 1.0000000000000004
value.(c1) = [0.0; 0.0; 0.0;;]
value.(c2) = [34.0; 25.000000000000004; 26.000000000000004; 29.000000000000004;;]
shadow_price.(c1) = [0.009730722628548101; 0.009929005639763044; 0.008065016159742477;;]
shadow_price.(c2) = [0.029411764705882353; 0.0; 0.0; 0.0;;]


4×1 Matrix{Float64}:
 0.029411764705882353
 0.0
 0.0
 0.0

In [104]:
su_rcot("dual", "abs", model_data);#, surplus=false);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 169.0
value.(p) = [1.0561836072199977, 0.9769985726736475, 1.1709128573052583, 0.0, 0.0]
value.(r) = [0.0, 0.0, 0.0, 0.0]
value.(profit_con) = [40.0, 57.0, 72.0]


In [103]:
#SU-NEO - LP
# Instantiate the model
dual = Model(GLPK.Optimizer)
(@isdefined dual) && (dual = nothing; dual = Model(GLPK.Optimizer));

# Set up the model
dual = Model(GLPK.Optimizer)

# Define the optimisation model
@variable(dual, p[1:size(sut.V,2)] >= 0)
@variable(dual, pii[1:size(sut.F,1)] >= 0)
@objective(dual, Min, sum(pii'*model_data.ϕ))
@constraint(dual, c1d, (model_data.V'-model_data.U)'*(p) .<= model_data.F'*pii)
@constraint(dual, c2d, (p'*model_data.e) .== sum(model_data.e))# or .==1?

# Run the model and show results
optimize!(dual)
model_solution(dual)
@show value.(p)
@show value.(pii)
@show shadow_price.(c1d)
@show shadow_price.(c2d);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 169.0
value.(p) = [1.1592841371453286, 1.2471962572194275, 0.16267777268079495, 0.0, 0.0]
value.(pii) = [4.946341463414633, 0.0, 0.0, 0.0]
shadow_price.(c1d) = [-1.0, -1.0, -1.0000000000000002]
shadow_price.(c2d) = [-1.0;;]


In [22]:
# SU-RCOT - MCP
# To set up our initial primal, we write:
compl = Model(PATHSolver.Optimizer)

act = size(model_data.V,1)
com = size(model_data.V,2)
fac = size(model_data.F,1)

# Define the optimisation model
@variable(compl, z_star[1:act] >= 0)
@variable(compl, p_star[1:com] >= 0)
@variable(compl, r_star[1:fac] >= 0)

@constraint(compl, c1, vec((model_data.pii + r_star)'*model_data.F - (p_star'*(model_data.V' - model_data.U))) ⟂ z_star)
@constraint(compl, c2, vec((model_data.V' - model_data.U)*(z_star) - model_data.e) ⟂ p_star)
@constraint(compl, c3, vec((model_data.ϕ - model_data.F*z_star)) ⟂ r_star)

# Solve
optimize!(compl)

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris


Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             1.5100e+02             0.0e+00 (f[    6])
    1     8     3     3 1.4960e+02  2.8e-02    1.5e+00 (f[    6])
    2    19     0     3 1.4958e+02  3.2e-04    1.4e+00 (f[    6])
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0    20     3 1.4958e+02           I 1.2e+00 1.3e+02 (f[    6])
    1     4    21     4 1.8600e+00  1.0e+00 SM 4.9e-01 1.0e+00 (f[    5])
    2     5    22     5 7.1801e-03  1.0e+00 SO 1.9e-01 4.7e-03 (f[    2])
    3     1    23     6 1.0146e-07  1.0e+00 SO 7.2e-04 4.4e-08 (f[    4])

Major Iterations. . . . 3
Minor Iterations. . . . 10
Restarts. . . . . . . . 0
Crash Iterations. . . . 2
Gradient Steps. . . . . 0
Function Evaluations. . 23
Gradient Evaluations. . 6
Basi

In [23]:
solution_summary(compl; verbose = true)

* Solver : Path 5.0.03

* Status
  Result count       : 1
  Termination status : LOCALLY_SOLVED
  Message from the solver:
  "The problem was solved"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 0.00000e+00
  Primal solution :
    p_star[1] : 2.20398e+00
    p_star[2] : 2.12643e+00
    p_star[3] : 2.39238e+00
    r_star[1] : 6.60676e-03
    r_star[2] : 3.89078e-03
    r_star[3] : 5.21122e-03
    r_star[4] : 5.40887e-03
    z_star[1] : 1.00000e+00
    z_star[2] : 1.00000e+00
    z_star[3] : 1.00000e+00

* Work counters
  Solve time (sec)   : 0.00000e+00


In [11]:
su_rcot("dual", "abs", model_data)

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 169.0
value.(p) = [1.0561836072199977, 0.9769985726736475, 1.1709128573052583, 0.0, 0.0]
value.(r) = [0.0, 0.0, 0.0, 0.0]
value.(profit_con) = [40.0, 57.0, 72.0]


A JuMP Model
Maximization problem with:
Variables: 9
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 9 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: GLPK
Names registered in the model: c1d, p, r

In [10]:
su_rcot("primal", "abs", model_data)

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 169.0
value.(var_con) = [1.0, 1.0, 1.0]
value.(demand_con) = [57.0, 79.0, 26.999999999999996, 4.0, 2.0]
value.(factor_con) = [34.16666666666667, 16.11111111111111, 2.2222222222222223, 30.476190476190474]


A JuMP Model
Minimization problem with:
Variables: 3
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 5 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 4 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 3 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: GLPK
Names registered in the model: c1, c2, z_star

In [212]:
print(compl)

In [9]:
value.(z_star)

3-element Vector{Float64}:
 1.0000000000107638
 1.0000000000103195
 1.0

In [10]:
value.(p_star)

5-element Vector{Float64}:
  0.5030856910415265
  0.780276068154548
  0.004556523736093214
  0.0006337864650413722
 39.36573642356929

In [11]:
value.(r_star)

4-element Vector{Float64}:
 0.0036396220584329784
 0.0009035815056565031
 7.701048377237372e-5
 0.0011696121734147977

MAybe something useful in this Julia opt cookbook? https://juliabook.chkwon.net/book/complementarity

A (good?) book on LCP/MCP: http://www-personal.umich.edu/%7Emurty/books/linear_complementarity_webbook/lcp-complete.pdf

There's an example for MCP in Julia in an answer [here](https://or.stackexchange.com/questions/9511/convert-mcp-model-from-gams-to-pyomo)

In [9]:
# SU-NEO: MCP (PATHSolver). Might not work because PATHSolver cannot deal with quadratic functions - which is included here as c4.
# To set up our initial primal, we write:
(@isdefined compl) && (compl = nothing; compl = Model(PATHSolver.Optimizer))

act = size(model_data.V,1)
com = size(model_data.V,2)
fac = size(model_data.F,1)

# Define the optimisation model
@variable(compl, z_star[1:act] >= 0)
@variable(compl, p_star[1:com] >= 0)
@variable(compl, pii_star[1:fac] >= 0)
@variable(compl, c_star >= 0)


@constraint(compl, c1, vec((model_data.V' - model_data.U)*(z_star) - model_data.e) ⟂ p_star)
@constraint(compl, c2, vec((pii_star)'*model_data.F - (p_star'*(model_data.V' - model_data.U))) ⟂ z_star)
@constraint(compl, c3, vec((model_data.ϕ - model_data.F*z_star)) ⟂ pii_star)
@constraint(compl, c4, dot(p_star',c_star,model_data.e) - dot(pii_star',model_data.ϕ) ⟂ c_star)

# Solve
optimize!(compl)

LoadError: Constraints of type MathOptInterface.VectorQuadraticFunction{Float64}-in-MathOptInterface.Complements are not supported by the solver.

If you expected the solver to support your problem, you may have an error in your formulation. Otherwise, consider using a different solver.

The list of available solvers, along with the problem types they support, is available at https://jump.dev/JuMP.jl/stable/installation/#Supported-solvers.

In [24]:
# SU-NEO: MCP
# PATHSolver cannot solve NL problem --> expansion factor c is problem
# To set up our initial primal, we write:
compl = MCPModel()

act = size(model_data.V,1)
com = size(model_data.V,2)
fac = size(model_data.F,1)

# Define the optimisation model
@variable(compl, z_star[1:act] >= 0)
@variable(compl, p_star[1:com] >= 0)
@variable(compl, pii_star[1:fac] >= 0)
@variable(compl, c_star >= 0)


@constraint(compl, c1, vec((model_data.V' - model_data.U)*(z_star) - c_star*model_data.e) ⟂ p_star)
@constraint(compl, c2, vec((pii_star)'*model_data.F - (p_star'*(model_data.V' - model_data.U))) ⟂ z_star)
@constraint(compl, c3, vec((model_data.ϕ - model_data.F*z_star)) ⟂ pii_star)
@constraint(compl, c4, dot(p_star',c_star,model_data.e) - dot(pii_star',model_data.ϕ) ⟂ c_star)#with scalar multiplication it must be dot product as described here: https://stackoverflow.com/questions/39079428/1-element-array-to-scalar-in-julia

#optimize!(compl)

c4 : [26 p_star[1]*c_star + 20 p_star[2]*c_star + 68 p_star[3]*c_star - 34 pii_star[1] - 25 pii_star[2] - 26 pii_star[3] - 29 pii_star[4], c_star] in MathOptInterface.Complements(2)

In [25]:
print(compl)

Is the reason for the below error that I don't provide initial values as in Problem 3 here? https://github.com/chkwon/Complementarity.jl/blob/master/MCP.md

In [26]:
status = solveMCP(compl, solver=:NLsolve)

LoadError: ArgumentError: range must be non-empty

In [10]:
m = MCPModel()

lb = zeros(4)
ub = Inf*ones(4)
items = 1:4
@variable(m, lb[i] <= x[i in items] <= ub[i])

@mapping(m, F1, 3*x[1]^2 + 2*x[1]*x[2] + 2*x[2]^2 + x[3] + 3*x[4] -6)
@mapping(m, F2, 2*x[1]^2 + x[1] + x[2]^2 + 3*x[3] + 2*x[4] -2)
@mapping(m, F3, 3*x[1]^2 + x[1]*x[2] + 2*x[2]^2 + 2*x[3] + 3*x[4] -1)
@mapping(m, F4, x[1]^2 + 3*x[2]^2 + 2*x[3] + 3*x[4] - 3)

@complementarity(m, F1, x[1])
@complementarity(m, F2, x[2])
@complementarity(m, F3, x[3])
@complementarity(m, F4, x[4])

setvalue(x[1], 1.25)
setvalue(x[2], 0.)
setvalue(x[3], 0.)
setvalue(x[4], 0.5)

status = solveMCP(m, solver=:NLsolve)
@show status

z = result_value(x)

@show z
@show Fz

LoadError: UndefVarError: `setvalue` not defined

In [28]:
@show value.(x)

┌ Warning: The model has been modified since the last call to `optimize!` (or `optimize!` has not been called yet). If you are iteratively querying solution information and modifying a model, query all the results first, then modify the model.
└ @ JuMP C:\Users\maximiko\.julia\packages\JuMP\jZvaU\src\optimizer_interface.jl:670


LoadError: OptimizeNotCalled()

In [30]:
(@isdefined m) && (m = nothing; m = MCPModel())

#m = MCPModel()

lb = zeros(4)
ub = Inf*ones(4)
items = 1:4
@variable(m, lb[i] <= x[i in items] <= ub[i])

@mapping(m, F1, 3*x[1]^2 + 2*x[1]*x[2] + 2*x[2]^2 + x[3] + 3*x[4] -6)
@mapping(m, F2, 2*x[1]^2 + x[1] + x[2]^2 + 3*x[3] + 2*x[4] -2)
@mapping(m, F3, 3*x[1]^2 + x[1]*x[2] + 2*x[2]^2 + 2*x[3] + 3*x[4] -1)
@mapping(m, F4, x[1]^2 + 3*x[2]^2 + 2*x[3] + 3*x[4] - 3)

@complementarity(m, F1, x[1])
@complementarity(m, F2, x[2])
@complementarity(m, F3, x[3])
@complementarity(m, F4, x[4])

#setvalue(x[1], 1.25)
#setvalue(x[2], 0.)
#setvalue(x[3], 0.)
#setvalue(x[4], 0.5)
set_start_value.(x, [1.25, 0., 0., 0.5])

status = solveMCP(m, solver=:NLsolve)
@show status


status = Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [1.25, 0.0, 0.0, 0.5]
 * Zero: [1.224744871126381, 0.0, -6.44459011190863e-20, 0.500000000228632]
 * Inf-norm of residuals: 0.000000
 * Iterations: 3
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 4
 * Jacobian Calls (df/dx): 4


Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [1.25, 0.0, 0.0, 0.5]
 * Zero: [1.224744871126381, 0.0, -6.44459011190863e-20, 0.500000000228632]
 * Inf-norm of residuals: 0.000000
 * Iterations: 3
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 4
 * Jacobian Calls (df/dx): 4

In [18]:
z = value.(x)

@show z
@show Fz

┌ Warning: The model has been modified since the last call to `optimize!` (or `optimize!` has not been called yet). If you are iteratively querying solution information and modifying a model, query all the results first, then modify the model.
└ @ JuMP C:\Users\maximiko\.julia\packages\JuMP\jZvaU\src\optimizer_interface.jl:670


LoadError: OptimizeNotCalled()

In [43]:
optimize!(compl)

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris


Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             0.0000e+00             0.0e+00 (f[    1])

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     1     1 0.0000e+00           I 0.0e+00 0.0e+00 (f[    1])

Major Iterations. . . . 0
Minor Iterations. . . . 0
Restarts. . . . . . . . 0
Crash Iterations. . . . 0
Gradient Steps. . . . . 0
Function Evaluations. . 1
Gradient Evaluations. . 1
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.000000
Residual. . . . . . . . 0.000000e+00


In [44]:
value.(z_star)

3-element Vector{Float64}:
 0.0
 0.0
 0.0

In [45]:
value.(p_star)

5-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0

In [46]:
value.(pii_star)

4-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0

In [40]:
model_data.e*c_star

5×1 Matrix{AffExpr}:
 57 c_star
 79 c_star
 27 c_star
 4 c_star
 2 c_star

In [33]:
p_star'*c_star*model_data.e

1×1 Matrix{QuadExpr}:
 57 p_star[1]*c_star + 79 p_star[2]*c_star + 27 p_star[3]*c_star + 4 p_star[4]*c_star + 2 p_star[5]*c_star

In [77]:
# NEO - MCP
# To set up our initial primal, we write:
compl = MCPModel()

# Define the optimisation model
@variable(compl, z_star[1:act,1:1] >= 0, base_name = "z*")
@variable(compl, p_star[1:com,1:1] >= 0, base_name = "p*")
@variable(compl, r_star[1:fac,1:1] >= 0, base_name = "r*")

@constraint(compl, c1[i in 1:fac, j in 1:act, k in 1:com], p_star ⟂  (su_rcot_data.pii[i]+r_star[i])*su_rcot_data.F[i,j] - p_star[k]*(su_rcot_data.V[j,k] - su_rcot_data.U[k,j]))
@constraint(compl, c1[i in 1:fac, j in 1:act, k in 1:com], p_star ⟂ (su_rcot_data.V' - su_rcot_data.U)*(z_star) - su_rcot_data.e)
@constraint(compl, c2[j in 1:act, k in 1:com], z_star' ⟂ (su_rcot_data.pii + r_star)'*su_rcot_data.F - (p_star'*(su_rcot_data.V' - su_rcot_data.U)))
@constraint(compl, c3[i in 1:fac, j in 1:act], r_star ⟂ (su_rcot_data.ϕ - su_rcot_data.F*z_star))


# To print the model:
# print(model) #or:
#latex_formulation(compl)

LoadError: MethodError: no method matching _build_complements_constraint(::JuMP.var"#_error#84"{Tuple{Symbol, Expr, Expr}, Symbol, LineNumberNode}, ::Matrix{VariableRef}, ::AffExpr)

[0mClosest candidates are:
[0m  _build_complements_constraint(::Function, ::AbstractArray{<:AbstractJuMPScalar}, [91m::AbstractArray{<:AbstractVariableRef}[39m)
[0m[90m   @[39m [35mJuMP[39m [90mC:\Users\maximiko\.julia\packages\JuMP\H2SWp\src\[39m[90m[4mcomplement.jl:12[24m[39m
[0m  _build_complements_constraint(::Function, ::AbstractArray{<:AbstractJuMPScalar}, [91m::AbstractArray{<:AbstractJuMPScalar}[39m)
[0m[90m   @[39m [35mJuMP[39m [90mC:\Users\maximiko\.julia\packages\JuMP\H2SWp\src\[39m[90m[4mcomplement.jl:42[24m[39m
[0m  _build_complements_constraint(::Function, [91m::AbstractJuMPScalar[39m, ::AbstractJuMPScalar)
[0m[90m   @[39m [35mJuMP[39m [90mC:\Users\maximiko\.julia\packages\JuMP\H2SWp\src\[39m[90m[4mcomplement.jl:58[24m[39m
[0m  ...


In [29]:
com

5

In [30]:
su_rcot_data.U

5×3 Matrix{Float64}:
  8.0  23.0  0.0
  0.0   7.0  9.0
 38.0   4.0  7.0
  2.0   4.0  0.0
  0.0   0.0  8.0

In [35]:
1 in [1:act]

false

In [40]:
for i=1:fac, j=1:act, k=1:com
    su_rcot_data.F[i,j]*(su_rcot_data.V[j,k] - su_rcot_data.U[k,j])
end

In [54]:
M = [(model_data.V'-model_data.U)' (-model_data.F')]

3×9 Matrix{Float64}:
  80.0   0.0  -38.0  -2.0  0.0  -17.5     -1.66667  -0.0        -6.19048
 -23.0  88.0   -4.0  -4.0  0.0  -16.6667  -6.94444  -0.740741   -4.7619
   0.0  -9.0   69.0  10.0  2.0   -0.0     -7.5      -1.48148   -19.5238

In [59]:
M*[p_star; r_star]

3-element Vector{AffExpr}:
 80 p*[1] - 38 p*[3] - 2 p*[4] - 17.5 r*[1] - 1.6666666666666667 r*[2] - 6.19047619047619 r*[4]
 -23 p*[1] + 88 p*[2] - 4 p*[3] - 4 p*[4] - 16.666666666666668 r*[1] - 6.944444444444445 r*[2] - 0.7407407407407407 r*[3] - 4.761904761904762 r*[4]
 -9 p*[2] + 69 p*[3] + 10 p*[4] + 2 p*[5] - 7.5 r*[2] - 1.4814814814814814 r*[3] - 19.523809523809522 r*[4]

In [67]:
M[1,:]*([p_star; r_star][1])

9-element Vector{AffExpr}:
 80 p*[1]
 0
 -38 p*[1]
 -2 p*[1]
 0
 -17.5 p*[1]
 -1.6666666666666667 p*[1]
 0
 -6.19047619047619 p*[1]

In [68]:
bar = [p_star; r_star][6]
M*bar

3×9 Matrix{AffExpr}:
 80 r*[1]   0         -38 r*[1]  -2 r*[1]  …  -6.19047619047619 r*[1]
 -23 r*[1]  88 r*[1]  -4 r*[1]   -4 r*[1]     -4.761904761904762 r*[1]
 0          -9 r*[1]  69 r*[1]   10 r*[1]     -19.523809523809522 r*[1]

In [65]:
M

3×9 Matrix{Float64}:
  80.0   0.0  -38.0  -2.0  0.0  -17.5     -1.66667  -0.0        -6.19048
 -23.0  88.0   -4.0  -4.0  0.0  -16.6667  -6.94444  -0.740741   -4.7619
   0.0  -9.0   69.0  10.0  2.0   -0.0     -7.5      -1.48148   -19.5238

In [70]:
j=1:(com+fac)

1:9

In [76]:
F[i in 1:act]

LoadError: UndefVarError: `i` not defined

In [72]:
@mapping(compl, F[i=1:act], M[i,j]*[p_star; r_star][j] for j=1:(com+fac))

LoadError: Unrecognized function ":" used in nonlinear expression.

You must register it as a user-defined function before building
the model. For example, replacing `N` with the appropriate number
of arguments, do:
```julia
model = Model()
register(model, ::, N, :, autodiff=true)
# ... variables and constraints ...
```


In [48]:
# SU-RCOT - MCP
compl = MCPModel()

@variable(compl, z_star[1:act] >= 0, base_name = "z*")
@variable(compl, p_star[1:com] >= 0, base_name = "p*")
@variable(compl, r_star[1:fac] >= 0, base_name = "r*")

@mapping(compl, profits[i in 1:fac, j in 1:act, k in 1:com],
        (su_rcot_data.pii[i]+r_star[i])*su_rcot_data.F[i,j] - 
        p_star[k]*(su_rcot_data.V[j,k] - su_rcot_data.U[k,j]))
@mapping(compl, demand[j in 1:act, k in 1:com],
        (su_rcot_data.V[j,k] - su_rcot_data.U[k,j])*z_star[j] -
        su_rcot_data.e[k])
@mapping(compl, factors[i in 1:fac, j in 1:act],
        su_rcot_data.ϕ[i] - su_rcot_data.F[i,j]*z_star[j])

@complementarity(compl, profits, z_star)
@complementarity(compl, demand, p_star)
@complementarity(compl, factors, r_star)

status = solveMCP(compl)
@show result_value.(z_star)
@show result_value.(p_star)
@show result_value.(r_star)

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             2.9819e+02             0.0e+00 (demand[(4,)])
    1    10     6     5 2.9815e+02  1.9e-03    3.0e+00 (demand[(4,)])
pn_search terminated: no progress.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0    23     2 2.9815e+02           I 2.7e+00 1.6e+02 (demand[(4,)])
    1     4    24     3 5.8470e+02  1.0e+00 SM 1.1e+00 5.7e+02 (demand[(2,)])
    2     1    25     4 1.4226e+03  1.0e+00 SD 4.3e-01 1.4e+03 (demand[(2,)])
    3     1    27     5 1.2425e+03  8.0e-01 SW 1.1e+00 1.2e+03 (demand[(2,)])
    4     1    28     6 2.0857e+03  1.0e+00 SD 4.3e-01 2.1e+03 (demand[(2,)])
    5     1    36     7 1.2864e+03  5.5e-02 SW 1.1e+00 1.3e+03 (demand[(2,)])
    6     1    37     8 2.1299e+03  1.0e+00 SD 4.3e-01 2.1e+03 (demand[(2,)])
  

4-element Vector{Float64}:
 0.018256352408369043
 0.0
 0.0
 0.0

In [49]:
print(compl)

In [24]:
# SU-RCOT: LCP
# To set up our initial primal, we write:
comp = MCPModel()

act = size(model_data.V,1)
com = size(model_data.V,2)
fac = size(model_data.F,1)

# Define the optimisation model
@variable(comp, z_star[1:act] >= 0)
@variable(comp, p_star[1:com] >= 0)
@variable(comp, r_star[1:fac] >= 0)


@constraint(comp, c1, vec((model_data.V' - model_data.U)*(z_star) - model_data.e) ⟂ p_star)
@constraint(comp, c2, vec((model_data.pii + r_star)'*model_data.F - (p_star'*(model_data.V' - model_data.U))) ⟂ z_star)
@constraint(comp, c3, vec((model_data.ϕ - model_data.F*z_star)) ⟂ r_star)

#optimize!(compl)

c3 : [-10 z_star[1] - 17 z_star[2] - 7 z_star[3] + 34, -4 z_star[1] - 21 z_star[3] + 25, -15 z_star[1] - 11 z_star[2] + 26, -2 z_star[1] - 15 z_star[2] - 12 z_star[3] + 29, r_star[1], r_star[2], r_star[3], r_star[4]] in MathOptInterface.Complements(8)

In [25]:
print(comp)

Is the reason for the below error that I don't provide initial values as in Problem 3 here? https://github.com/chkwon/Complementarity.jl/blob/master/MCP.md

In [26]:
status = solveMCP(comp, solver=:PATH)

:Solved

In [30]:
optimize!(comp)

LoadError: NoOptimizer()

In [43]:
status

:Solved

In [44]:
z_star

3-element Vector{VariableRef}:
 z_star[1]
 z_star[2]
 z_star[3]

In [45]:
@show result_value(z_star)
@show result_value(p_star)
@show result_value(r_star)

LoadError: MethodError: no method matching result_value(::Vector{VariableRef})

[0mClosest candidates are:
[0m  result_value([91m::VariableRef[39m)
[0m[90m   @[39m [33mComplementarity[39m [90mC:\Users\maximiko\.julia\packages\Complementarity\NbsQM\src\[39m[90m[4mmcp.jl:372[24m[39m


In [29]:
optimize!

optimize! (generic function with 1 method)

In [35]:
solution_summary(comp)

┌ Warning: The model has been modified since the last call to `optimize!` (or `optimize!` has not been called yet). If you are iteratively querying solution information and modifying a model, query all the results first, then modify the model.
└ @ JuMP C:\Users\maximiko\.julia\packages\JuMP\jZvaU\src\optimizer_interface.jl:650
┌ Warning: The model has been modified since the last call to `optimize!` (or `optimize!` has not been called yet). If you are iteratively querying solution information and modifying a model, query all the results first, then modify the model.
└ @ JuMP C:\Users\maximiko\.julia\packages\JuMP\jZvaU\src\optimizer_interface.jl:650
┌ Warning: The model has been modified since the last call to `optimize!` (or `optimize!` has not been called yet). If you are iteratively querying solution information and modifying a model, query all the results first, then modify the model.
└ @ JuMP C:\Users\maximiko\.julia\packages\JuMP\jZvaU\src\optimizer_interface.jl:650
┌ Warning: Th

* Solver : No optimizer attached.

* Status
  Result count       : 0
  Termination status : OPTIMIZE_NOT_CALLED
  Message from the solver:
  "optimize not called"

* Candidate solution (result #1)
  Primal status      : NO_SOLUTION
  Dual status        : NO_SOLUTION

* Work counters


In [34]:
@show value.(z_star)

┌ Warning: The model has been modified since the last call to `optimize!` (or `optimize!` has not been called yet). If you are iteratively querying solution information and modifying a model, query all the results first, then modify the model.
└ @ JuMP C:\Users\maximiko\.julia\packages\JuMP\jZvaU\src\optimizer_interface.jl:670


LoadError: OptimizeNotCalled()

In [20]:
# SU-NST - MCP
# To set up our initial primal, we write:
compl = Model(PATHSolver.Optimizer)

act = size(model_data.V,1)
com = size(model_data.V,2)
fac = size(model_data.F,1)

# Define the optimisation model
@variable(compl, z_star[1:act] >= 0)
@variable(compl, p_star[1:com] >= 0)
@variable(compl, pii_star[1:fac] >= 0)

@constraint(compl, c1, vec(pii_star'*model_data.F - (p_star'*(model_data.V' - model_data.U))) ⟂ z_star)
@constraint(compl, c2, vec((model_data.V' - model_data.U)*(z_star) - model_data.e) ⟂ p_star)
@constraint(compl, c3, vec((model_data.ϕ - model_data.F*z_star)) ⟂ pii_star)

# Solve
optimize!(compl)

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris


Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             1.5100e+02             0.0e+00 (f[    6])
    1    12     5     3 1.5099e+02  5.4e-05    1.5e+00 (f[    6])
    2    22     3     5 1.5095e+02  1.9e-03    1.4e+00 (f[    6])
    3    23     5     6 3.1214e+00  1.0e+00    1.2e+00 (f[    2])
    4    24     4    10 1.2917e-03  1.0e+00    3.1e-02 (f[    4])
    5    25     0    10 3.3275e-10  1.0e+00    1.3e-05 (f[    2])
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0    26     6 3.3275e-10           I 3.3e-12 2.2e-10 (f[    2])

Major Iterations. . . . 0
Minor Iterations. . . . 0
Restarts. . . . . . . . 0
Crash Iterations. . . . 5
Gradient Steps. . . . . 0
Function Evaluations. . 26
Gradient Evaluations. . 6
Basis Time. . . . . . . 0.000

In [21]:
solution_summary(compl; verbose = true)

* Solver : Path 5.0.03

* Status
  Result count       : 1
  Termination status : LOCALLY_SOLVED
  Message from the solver:
  "The problem was solved"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 0.00000e+00
  Primal solution :
    p_star[1] : 1.41298e-02
    p_star[2] : 1.42032e-02
    p_star[3] : 1.34631e-02
    pii_star[1] : 1.68572e-02
    pii_star[2] : 9.91560e-03
    pii_star[3] : 1.32931e-02
    pii_star[4] : 1.38025e-02
    z_star[1] : 1.00000e+00
    z_star[2] : 1.00000e+00
    z_star[3] : 1.00000e+00

* Work counters
  Solve time (sec)   : 0.00000e+00


In [31]:
su_rcot_model = su_rcot("primal", "abs", model_data)

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 261.9
value.(var_con) = [1.0, 1.0, 1.0]
value.(demand_con) = [26.0, 20.0, 68.0]
value.(factor_con) = [34.0, 25.0, 26.0, 29.0]


A JuMP Model
Minimization problem with:
Variables: 3
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 3 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 4 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 3 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: GLPK
Names registered in the model: c1, c2, z_star

In [49]:
io_rcot_model = io_rcot("dual", "abs", model_data)

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 261.9
value.(p) = [2.1984456937063546, 2.120861917519434, 2.3871054943124417]
value.(r) = [0.0, 0.0, 0.0, 0.0]
value.(profit_con) = [99.18888888888888, 40.591743119266056, 122.11936799184505]


A JuMP Model
Maximization problem with:
Variables: 7
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 7 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: GLPK
Names registered in the model: c1d, p, r